## Harmonic Oscillator

$$m\frac{d^2u}{dt^2} + \mu\frac{du}{dt} + k u =0,\quad u(0)=1,\quad \frac{du}{dt}(0) = 0,\quad t\in[0, 1]$$

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
def set_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        
def to_tensor(x):
    return torch.tensor(x).float().view(-1, 1).to(device)

def to_array(x):
    return x.view(-1).cpu().detach().numpy()

def gradient(y, x):
    return torch.autograd.grad(y, x, grad_outputs=torch.ones_like(y),
                               create_graph=True, retain_graph=True)[0]

In [8]:
## Equation parameters:
d, w0 = 2, 20       ## reference value w0 = 20
mu, k = 2*d, w0**2

t_min, t_max, t_size = 0, 1, 101
t = np.linspace(t_min, t_max, t_size)       # (t_size,)

n_pde = 101
## Domain: u_tt + mu * u_t + k * u
t_pde = np.random.rand(n_pde)*(t_max - t_min) + t_min
t_pde = to_tensor(t_pde)

t_ic = to_tensor(0)
u_ic = to_tensor(0)
du_ic = to_tensor(1)

In [ ]:
class PINN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(1, 50),  nn.Tanh(),
            nn.Linear(50, 50), nn.Tanh(),
            nn.Linear(50, 50), nn.Tanh(),
            nn.Linear(50, 1),
        )

    def forward(self, t):
        return self.net(t)  # (N, 1)
    
    def residual_loss(self, t, mu=mu, k=k):
        t.requires_grad = True
        u = self.forward(t)
        u_t = gradient(u, t)
        u_tt = gradient(u_t, t)
        residual = u_tt + mu * u_t + k * u
        return torch.mean(residual**2)
    
    def ic_du_loss(self, t):
        t.requires_grad = True
        u = self.forward(t)
        u_t = gradient(u, t)
        return torch.mean((u_t - torch.full_like(t, 0))**2)
    
    def ic_u_loss(self, t):
        u = self.forward(t)
        return torch.mean((u - torch.full_like(t, 1))**2)

In [ ]:
## Training
n_epochs = 10000
learning_rate = 1e-3

model = PINN().to(device)
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1000, gamma=0.985)

for epoch in range(1, n_epochs + 1):
    model.train()
    loss_pde = model.residual_loss(t_pde)
    loss_ic_du = model.ic_du_loss(t_ic)
    loss_ic_u = model.ic_u_loss(t_ic)

    loss = loss_pde + loss_ic_du + loss_ic_u
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    scheduler.step()

    if epoch % (n_epochs // 10) == 0:
        print(f"[{epoch:5d}/{n_epochs}] (lr: {scheduler.get_last_lr()[0]:.2e}) "
              f"loss: {loss.item():.2e} "
              f"(pde: {loss_pde.item():.2e} ic_du: {loss_ic_du.item():.2e} "
              f"ic_u: {loss_ic_u.item():.2e})")